In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import metrics as metrics

2023-12-12 14:43:13.634954: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 14:43:13.636707: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-12 14:43:13.662669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 14:43:13.662699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 14:43:13.663457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import pathlib
data_dir = pathlib.Path("/home/user/.keras/datasets/CUB_200_2011_backup_removed_background/images")

In [3]:
batch_size = 32
img_height = 180
img_width = 180

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 11788 files belonging to 200 classes.
Using 9431 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 11788 files belonging to 200 classes.
Using 2357 files for validation.


In [7]:
class_names = train_ds.class_names

In [8]:
#Configure the dataset for performance - I don't know what this does
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
#a very simple model I just got on the internet
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
epochs=7
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/7
295/295 [==============================] - 34s 111ms/step - loss: 5.2396 - accuracy: 0.0077 - val_loss: 5.0088 - val_accuracy: 0.0208
Epoch 2/7
295/295 [==============================] - 33s 112ms/step - loss: 4.5735 - accuracy: 0.0634 - val_loss: 4.5897 - val_accuracy: 0.0755
Epoch 3/7
295/295 [==============================] - 35s 118ms/step - loss: 3.3991 - accuracy: 0.2358 - val_loss: 4.5607 - val_accuracy: 0.0976
Epoch 4/7
295/295 [==============================] - 34s 114ms/step - loss: 1.8657 - accuracy: 0.5379 - val_loss: 5.7007 - val_accuracy: 0.0904
Epoch 5/7
295/295 [==============================] - 33s 112ms/step - loss: 0.7247 - accuracy: 0.8112 - val_loss: 7.2714 - val_accuracy: 0.0861
Epoch 6/7
295/295 [==============================] - 33s 112ms/step - loss: 0.2370 - accuracy: 0.9389 - val_loss: 9.7390 - val_accuracy: 0.0789
Epoch 7/7
295/295 [==============================] - 33s 112ms/step - loss: 0.1056 - accuracy: 0.9726 - val_loss: 10.8962 - val_accuracy